In [13]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import os

In [14]:
conf = SparkConf()
conf.set('spark.jars', 'postgresql-42.7.3.jar')

In [15]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("local data ingestion") \
    .config(conf=conf) \
    .getOrCreate()


In [16]:
df = spark.read.option("header","true").parquet('green_tripdata_2024-01.parquet')

In [17]:
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2024-01-01 00:46:55|  2024-01-01 00:58:25|                 N|         1|         236|         239|              1|         1.98|       12.8|  1.0|    0.

In [18]:
df.write.format("jdbc") \
 .options(url="jdbc:postgresql://172.19.0.2:5432/newyorktaxi",
          dbtable="public.green_tripdata",
          user="root",
          password="root",
          driver="org.postgresql.Driver") \
 .mode('overwrite') \
 .save()